In [ ]:
# Copyright 2021 Xilinx, Inc.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

ILA Basic Trigger Example
=========================

Description
-----------
This demo shows how to do four things:
1. Connect to a Versal target device via ChipScope Server (cs_server)
   and Hardware Server (hw_server)
2. Program a Versal target device using a design PDI file
3. Run a basic trigger on the ILA core
4. Retrieve and dump the data buffer captured by the ILA core

Requirements
------------
The following is required to run this demo:
1. Local or remote access to a Versal device
2. 2021.1 cs_server and hw_server applications
3. Python 3.8 environment
4. A clone of the chipscopy git enterprise repository:
   - https://gitenterprise.xilinx.com/chipscope/chipscopy

---

## Step 1 - Set up environment

In [1]:
import os
from enum import Enum
from chipscopy import get_examples_dir_or_die, null_callback
from statistics import mean, median
from chipscopy.api.ila import export_waveform, get_waveform_probe_data, get_waveform_data

In [2]:
# Specify locations of the running hw_server and cs_server below.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")
BOARD = os.getenv("HW_SERVER_BOARD", "vck190/production/2.0")

In [3]:
EXAMPLES_DIR = get_examples_dir_or_die()
PDI_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.pdi"
assert os.path.isfile(PDI_FILE)
LTX_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.ltx"
assert os.path.isfile(LTX_FILE)

In [4]:
print(f"HW_URL={HW_URL}")
print(f"CS_URL={CS_URL}")
print(f"PDI={PDI_FILE}")
print(f"LTX={LTX_FILE}")

HW_URL=TCP:ibert-1:3121
CS_URL=TCP:localhost:3042
PDI=C:\Users\jant\PycharmProjects\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi
LTX=C:\Users\jant\PycharmProjects\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.ltx


## Step 2 - Create a session and connect to the server(s)
Here we create a new session and print out some versioning information for diagnostic purposes.
The session is a container that keeps track of devices and debug cores.

In [5]:
import chipscopy
from chipscopy import create_session, report_versions

In [6]:
print(f"Using chipscopy api version: {chipscopy.__version__}")
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

Using chipscopy api version: 2021.1.1613109357


## Step 3 - Get our device from the session

In [7]:
# Use the first available device and setup its debug cores
if len(session.devices) == 0:
    raise ValueError("No devices detected")
print(f"Device count: {len(session.devices)}")
versal_device = session.devices[0]

Device count: 1


## Step 4 - Program the device with our example PDI

In [8]:
print(f"Programming {PDI_FILE}...")
versal_device.program(PDI_FILE)

Programming C:\Users\jant\PycharmProjects\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi...


Output()

## Step 5 - Detect Debug Cores

In [9]:
print(f"Discovering debug cores...")
versal_device.discover_and_setup_cores(ltx_file=LTX_FILE)

Discovering debug cores...


In [10]:
ila_count = len(versal_device.ila_cores)
print(f"\nFound {ila_count} ILA cores in design")


Found 4 ILA cores in design


In [11]:
if ila_count == 0:
    print("No ILA core found! Exiting...")
    raise ValueError("No ILA cores detected")

In [12]:
# List all detected ILA Cores
ila_cores = versal_device.ila_cores
for index, ila_core in enumerate(ila_cores):
    print(f"    ILA Core #{index}: NAME={ila_core.name}, UUID={ila_core.core_info.uuid}")

    ILA Core #0: NAME=ks_demo_i/axis_ila_dma, UUID=1CBBF19A124D562B8B39E5D39BB10BE3
    ILA Core #1: NAME=ks_demo_i/axis_ila_bram, UUID=20E8B004FC865B87883C4CCDC77D830D
    ILA Core #2: NAME=ks_demo_i/ila_fast_counter_0, UUID=8F904C7908B25897BEAB95475B844358
    ILA Core #3: NAME=ks_demo_i/ila_slow_counter_0, UUID=CA49247E0AB35CCBB6093B473F933C0E


### Some ways to query debug cores

In [13]:
# Get the ila cores matching a given name. filter_by returns a list, even if just one item is present.
ila_by_name = versal_device.ila_cores.filter_by(name="ks_demo_i/ila_slow_counter_0")[0]
# Get ILA by uuid:
ila_by_uuid = versal_device.ila_cores.filter_by(uuid="CA49247E0AB35CCBB6093B473F933C0E")[0]
# This also works to get an ila by index:
ila_by_index = versal_device.ila_cores[3]

In [14]:
assert ila_by_name is ila_by_uuid
assert ila_by_name is ila_by_index

In [15]:
my_ila = ila_by_name

In [16]:
print(f"USING ILA: {my_ila.name}")

USING ILA: ks_demo_i/ila_slow_counter_0


## Step 6 - Get Information for this ILA Core


In [17]:
from pprint import pformat

In [18]:
print("\nILA Name:", my_ila.name)
print("\nILA Core Info", my_ila.core_info)
print("\nILA Static Info", my_ila.static_info)
print("\nILA ports:", pformat(my_ila.ports, 2))
print("\nILA probes:", pformat(my_ila.probes, 2))


ILA Name: ks_demo_i/ila_slow_counter_0

ILA Core Info { 'core_major_ver': 1,
  'core_minor_ver': 0,
  'core_type': 1,
  'drv_ver': 4,
  'tool_major_ver': 19,
  'tool_minor_ver': 1,
  'uuid': 'CA49247E0AB35CCBB6093B473F933C0E'}

ILA Static Info { 'data_depth': 1024,
  'data_width': 72,
  'has_advanced_trigger': True,
  'has_capture_control': False,
  'has_trig_in': True,
  'has_trig_out': True,
  'match_unit_count': 40,
  'port_count': 10}

ILA ports: [ ILAPort(bit_width=1, data_bit_index=0, index=0, is_trigger=True, is_data=True, match_unit_count=4, match_unit_type=<ILAMatchUnitType.LARGE: 2>),
  ILAPort(bit_width=1, data_bit_index=1, index=1, is_trigger=True, is_data=True, match_unit_count=4, match_unit_type=<ILAMatchUnitType.LARGE: 2>),
  ILAPort(bit_width=1, data_bit_index=2, index=2, is_trigger=True, is_data=True, match_unit_count=4, match_unit_type=<ILAMatchUnitType.LARGE: 2>),
  ILAPort(bit_width=1, data_bit_index=3, index=3, is_trigger=True, is_data=True, match_unit_count=4, ma

## Step 7 - Trigger on a probe value

This ILA core is connected to a counter on probe 'ks_demo_i/slow_counter_0_Q_1'.
Here we will trigger on the hex value equal to 0xXXXX_ABCD'

Once the buffer is full of captured data, status['is_full'] == True

In [19]:
print("Running trigger on probe 'ks_demo_i/slow_counter_0_Q_1' ...")

Running trigger on probe 'ks_demo_i/slow_counter_0_Q_1' ...


In [20]:
# Set probe compare value to hex 0xXXXX_ABCD
my_ila.set_probe_trigger_value("ks_demo_i/slow_counter_0_Q_1", ["==", "0xXXXX_ABCD"])

In [21]:
# default values: trigger position in middle of window, trigger condition AND.
my_ila.run_basic_trigger(window_count=3, window_size=8, trigger_position=0)

In [22]:
my_ila.refresh_status()
print("\nILA Status", my_ila.status)


ILA Status { 'capture_state': <ILAState.IDLE: 0>,
  'is_armed': False,
  'is_full': True,
  'samples_captured': 0,
  'samples_requested': 8,
  'trigger_position_requested': 0,
  'windows_captured': 3,
  'windows_requested': 3}


## Step 8 - Upload captured waveform
Wait at most half a minutes, for ILA to trigger and capture data.

In [23]:
my_ila.monitor_status(max_wait_minutes=0.5)
my_ila.upload()
if not my_ila.waveform:
    print("\nUpload failed!")

### Print out probe 'ks_demo_i/slow_counter_0_Q_1' data, in CSV format.

Below is a snippet that gets values for probe 'ks_demo_i/slow_counter_0_Q_1' in a list.
The list contains namedtuples for each probe 'ks_demo_i/slow_counter_0_Q_1' sample captured.

In [24]:
print("\nDump signal ks_demo_i/slow_counter_0_Q_1 waveform to stdout in CSV format:\n")
export_waveform(my_ila.waveform, export_format="CSV", probe_names=["ks_demo_i/slow_counter_0_Q_1"])


Dump signal ks_demo_i/slow_counter_0_Q_1 waveform to stdout in CSV format:

Sample in Buffer,Sample in Window,TRIGGER,ks_demo_i/slow_counter_0_Q_1
Radix - UNSIGNED,UNSIGNED,UNSIGNED,HEX
0,0,1,221DABCD
1,1,0,221DABCE
2,2,0,221DABCF
3,3,0,221DABD0
4,4,0,221DABD1
5,5,0,221DABD2
6,6,0,221DABD3
7,7,0,221DABD4
8,0,1,221EABCD
9,1,0,221EABCE
10,2,0,221EABCF
11,3,0,221EABD0
12,4,0,221EABD1
13,5,0,221EABD2
14,6,0,221EABD3
15,7,0,221EABD4
16,0,1,221FABCD
17,1,0,221FABCE
18,2,0,221FABCF
19,3,0,221FABD0
20,4,0,221FABD1
21,5,0,221FABD2
22,6,0,221FABD3
23,7,0,221FABD4


### Get samples for probe 'ks_demo_i/slow_counter_0_Q_1'. Dump in decimal and hex.

The waveform data is put into a sorted dict.
First 4 entries in sorting order are: trigger, sample index, window index, window sample index.
The comes probe values. In this case only the requested probe 'ks_demo_i/slow_counter_0_Q_1'.

In [25]:
print("\nGet ILA captured values for ks_demo_i/slow_counter_0_Q_1, dump in decimal and hex.")
samples = get_waveform_data(
    my_ila.waveform,
    ["ks_demo_i/slow_counter_0_Q_1"],
    include_trigger=True,
    include_sample_info=True,
)
for trigger, sample_index, window_index, window_sample_index, value in zip(*samples.values()):
    trigger = "<-- Trigger" if trigger else ""
    print(
        f"Window:{window_index}  Window Sample:{window_sample_index}  {value:10}  0x{value:08X} {trigger}"
    )


Get ILA captured values for ks_demo_i/slow_counter_0_Q_1, dump in decimal and hex.
Window:0  Window Sample:0   572369869  0x221DABCD <-- Trigger
Window:0  Window Sample:1   572369870  0x221DABCE 
Window:0  Window Sample:2   572369871  0x221DABCF 
Window:0  Window Sample:3   572369872  0x221DABD0 
Window:0  Window Sample:4   572369873  0x221DABD1 
Window:0  Window Sample:5   572369874  0x221DABD2 
Window:0  Window Sample:6   572369875  0x221DABD3 
Window:0  Window Sample:7   572369876  0x221DABD4 
Window:1  Window Sample:0   572435405  0x221EABCD <-- Trigger
Window:1  Window Sample:1   572435406  0x221EABCE 
Window:1  Window Sample:2   572435407  0x221EABCF 
Window:1  Window Sample:3   572435408  0x221EABD0 
Window:1  Window Sample:4   572435409  0x221EABD1 
Window:1  Window Sample:5   572435410  0x221EABD2 
Window:1  Window Sample:6   572435411  0x221EABD3 
Window:1  Window Sample:7   572435412  0x221EABD4 
Window:2  Window Sample:0   572500941  0x221FABCD <-- Trigger
Window:2  Window

### Print mean and median for probe 'ks_demo_i/slow_counter_0_Q_1' values.

In [26]:
mean_value = mean(samples["ks_demo_i/slow_counter_0_Q_1"])
print(f"\nMean Value: {mean_value}")


Mean Value: 572435408.5


In [27]:
values = get_waveform_probe_data(my_ila.waveform, "ks_demo_i/slow_counter_0_Q_1")
median_value = median(values)
print(f"Median Value: {median_value}")

Median Value: 572435408.5
